In [1]:
import torch.nn as nn
from utils.run import run
from utils.plot import loss_curve

In [2]:
# information
filename     = "PDAC_Train_byCCL_mut_drug_1fold_package"

In [3]:
#hyperparameter
model_name = "Mut_Drug_Model"
splitType= 'byCCL'
valueMultiply=1

mut_encode_dim =[33,22,14] 
drug_encode_dim =[116,81,56] #
mut_encode_dim_save = f'(33_22_14)'
drug_encode_dim_save= f'(116_81_56)'
activation_func = nn.ReLU()  # ReLU activation function
activation_func_final = nn.Sigmoid()# sigmoid()
dense_layer_dim = 200 # MLPDim
batch_size = 1000
num_epoch = 200 # for k fold CV 
patience = 20
warmup_iters = 150
Decrease_percent = 0.99
continuous = True
learning_rate=1e-05
criterion = nn.MSELoss()
# loss need to be devided with valueMultiply^2 if lossfunction is MSE, if is L1 than devide with valueMultiply
# NNoutput need to be multiplied  with valueMultiply if activatefunction is sigmoid(), if ReLU() don't multiply

hyperparameter_print = f' model_name ={model_name}\n splitType ={splitType}\n valueMultiply ={valueMultiply}\n mut_encode_dim ={mut_encode_dim}\n drug_encode_dim ={drug_encode_dim}\n activation_func = {activation_func}\n activation_func_final = {activation_func_final}\n dense_layer_dim = {dense_layer_dim}\n batch_size = {batch_size}\n num_epoch = {num_epoch}\n patience = {patience}\n warmup_iters = {warmup_iters}\n Decrease_percent = {Decrease_percent}\n continuous ={continuous}\n learning_rate = {learning_rate}\n criterion ={criterion}'
hyperparametersave = f'data*{valueMultiply}_{splitType}_Mut{mut_encode_dim_save}Drug{drug_encode_dim_save}ActFun{activation_func}ActFunFinl{activation_func_final}MLPDim{dense_layer_dim}BatcSiz{batch_size}Epoc{num_epoch}Patien{patience}warmup{warmup_iters}Decre%{Decrease_percent}LR{learning_rate}{criterion}'
hyperparameters = {
    "model_name": model_name,
    "splitType": splitType,
    "valueMultiply": valueMultiply,
    "mut_encode_dim": mut_encode_dim,
    "drug_encode_dim": drug_encode_dim,
    "activation_func": activation_func,  # ReLU activation function
    "activation_func_final": activation_func_final,  # Sigmoid activation function
    "dense_layer_dim": dense_layer_dim,
    "batch_size": batch_size,
    "num_epoch": num_epoch,  # for k fold CV
    "patience": patience,
    "warmup_iters": warmup_iters,
    "Decrease_percent": Decrease_percent,
    "continuous": continuous,
    "learning_rate": learning_rate,
    "criterion": criterion,
    "hyperparameter_print": hyperparameter_print,
    "hyperparametersave": hyperparametersave
}


In [4]:
loss_values_train, loss_values_validation, best_epoch, best_val_loss = run(hyperparameters,filename)

Training on device cuda.


Datasets successfully loaded.
(1442, 166)
(480, 36)
(480, 1442)
(480, 1442)
torch.Size([692160, 36])
torch.Size([692160, 166])
torch.Size([692160])
data_AUC[1] tensor(0.9036, device='cuda:0')
0.82050472
id_unrepeat_train (389,)
id_unrepeat_val (43,)
id_unrepeat_test (48,)
id_unrepeat_train_val (432,)
id_train.shape (560938,)
id_val.shape (62006,)
id_test.shape (69216,)


100%|██████████| 3/3 [00:00<00:00, 8479.05it/s]


create dataset finished


100%|██████████| 3/3 [00:00<00:00, 16935.28it/s]


create dataset finished


100%|██████████| 3/3 [00:00<00:00, 1277.32it/s]

create dataset finished
State_dict loaded successfully.


In [ ]:
loss_curve(model_name, loss_values_train, loss_values_validation, best_epoch, best_val_loss, ylim_top=0.035)

predict PDAC

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch.nn.init as init
from sklearn.model_selection import KFold
import copy
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from utils.load_data import load_ccl, load_AUC_matrix, load_drug
from utils.data_repeat_to_instance import repeat_data
from utils.Mut_Drug_Model import Mut_Drug_Model
from utils.split_data_id import split_id
from utils.create_dataloader import create_dataset
from utils.train import train, evaluation
from utils.correlation import correlation_func
from utils.plot import loss_curve, correlation_density
from utils.tools import get_data_value_range

In [ ]:
#load data
data_drug, drug_names  = load_drug("./PDAC_drug_MACCS.txt")
data_mut, gene_names_mut,ccl_names_mut  = load_ccl("./PDAC_matched_genenames_TCGA_CCLE_binary_mutation.txt")

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

In [ ]:
print(data_drug.shape)
print(data_mut.shape)

In [ ]:
data_mut, data_drug, _ =repeat_data(data_mut, data_drug, 1, splitType= 'byCCL', num_ccl=50, num_drug=8, device=device) 

In [ ]:
print(data_mut)

In [ ]:
predict_dataset = TensorDataset(data_mut,data_drug)
predict_loader = DataLoader(predict_dataset, shuffle=False)

In [ ]:
# train_loss, train_targets, train_outputs = evaluation(model, activation_func_final, epoch, num_epoch, loss_values_validation, criterion, valueMultiply, train_loader, device, correlation='train', kfoldCV = None)

In [ ]:
mut_encode_dim =[33,22,14] 
drug_encode_dim =[116,81,56] #
activation_func = nn.ReLU()  # ReLU activation function
activation_func_final = nn.Sigmoid()# sigmoid()
dense_layer_dim = 200

In [ ]:
model = Mut_Drug_Model(mut_encode_dim, drug_encode_dim, activation_func, activation_func_final, dense_layer_dim, device,
                        num_mut_features=data_mut.shape[1], num_drug_features=data_drug.shape[1],
                        TCGA_pretrain_weight_path= "/root/Winnie/PDAC/results/Encoder_tcga_mut_33_22_14_best_loss_0.0078.pt").to(device=device)

model.load_state_dict(torch.load("./results/PDAC_Train_byCCL_mut_drug_1fold_BestValLoss_0.007486213509525571_MutEncoDim(33_22_14)_DrugEncoDim(116_81_56)_ActivaFuncReLU()_ActivaFuncFinalSigmoid()_MLPDim200_BatchSize1000_NumEpoch200_Patien20_warmupiters150_DecresPercen0.99_LearnRate1e-05.pt"))  

In [ ]:
predict_outputs = []
model.eval()
model.requires_grad = False
with torch.no_grad():
    for batch_idx,inputs in enumerate(predict_loader):
        mut,drug, target = inputs[0].to(device=device),inputs[1].to(device=device), inputs[-1].to(device=device)
        outputs = model(mut.to(torch.float32), drug.to(torch.float32))
        predict_outputs.append(outputs.detach().cpu().numpy().reshape(-1))
            

In [ ]:
np.shape(np.array(predict_outputs).reshape(-1))

In [ ]:
predict_outputs_list = [(item[0]) for item in predict_outputs]
predict_outputs_list

In [ ]:
patient_ID = [item for item in ccl_names_mut for _ in range(8)]
print((patient_ID))

drug_names_instance = drug_names * 50
drug_names_instance


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(6, 5))
sns.kdeplot(predict_outputs_list, color='red', label='predicted_AUC')
plt.xlabel('AUC values',fontsize=15)
# Set the y-axis label 
plt.ylabel('Density',fontsize=15)
# Set the title of the plot
plt.title('Density Plot of predicted_AUC values',fontsize=15)
plt.legend(fontsize=12)
plt.show()
get_data_value_range(predict_outputs_list,"GroundTruth_AUC")

In [ ]:
import random

# Mapping labels to colors
# Generate random colors for each label
# label_to_color = {'GEMCITABINE': 'red', 'PACLITAXEL': 'blue', 'TS-1': 'green', 'Cisplatin':'yellow','ELOXATIN':'purple',
#                   'FLUOROURACIL':'black','LEUCOVORIN':'light blue','IRINOTECAN':'brown'}
# Get unique labels
unique_labels = list(set(drug_names_instance))

# Define a list of colors (ensure they are valid color names or RGB tuples)
color_list = ['red', 'blue', 'green', 'gray', 'purple', 'black', 'orange','brown']

# Map each label to a color
label_to_color = {label: color_list[i % len(color_list)] for i, label in enumerate(unique_labels)}

# Assign colors based on labels
colors = [label_to_color[label] for label in drug_names_instance]

# Plotting the dot plot
plt.figure(figsize=(8, 6))
plt.scatter(range(len(predict_outputs_list)), predict_outputs_list, c=colors, alpha=0.7)

# Adding a legend
for label in unique_labels:
    plt.scatter([], [], c=label_to_color[label], label=label)

plt.legend(title="Labels",fontsize=11)
plt.title('Dot Plot',fontsize=15)
plt.xlabel('samples',fontsize=15)
plt.ylabel('Predicted AUC',fontsize=15)
plt.show()

In [ ]:
import csv
with open('./results/PDAC_predict_result.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Patient_ID', 'Drug_name', 'Predicted_AUC'])  # Write header
    for i in range(len(patient_ID)):
        writer.writerow([patient_ID[i], drug_names_instance[i], predict_outputs_list[i]])